# Solving optimization problems with the kernel Trust-Region algorithm 

In this demo we want to demonstrate how to use the kernel TR algorithm to solve a 2D optimization problem. 
The problem we want to adress is taken from the pyMOR tutorial: Model order reduction for PDE-constrained optimization problems. We first import some standard python libraries. 


In [1]:
from pymor.basic import *
import numpy as np
import math as m
import time 

We define the optimization problem as in Chapter 7.1 of the thesis. 


In [2]:
from problems import linear_problem

problem = linear_problem()
mu_bar = problem.parameters.parse([np.pi/2,np.pi/2])
fom, data = discretize_stationary_cg(problem, diameter=1/100, mu_energy_product=mu_bar)
parameter_space = fom.parameters.space(0, np.pi)

00:03 DiffusionOperatorP1: Calculate gradients of shape functions transformed by reference map ...
00:03 DiffusionOperatorP1: Calculate all local scalar products between gradients ...
00:03 DiffusionOperatorP1: Determine global dofs ...
00:03 DiffusionOperatorP1: Boundary treatment ...
00:04 DiffusionOperatorP1: Assemble system matrix ...
00:04 DiffusionOperatorP1: Calculate gradients of shape functions transformed by reference map ...
00:04 DiffusionOperatorP1: Calculate all local scalar products between gradients ...
00:04 DiffusionOperatorP1: Determine global dofs ...
00:04 DiffusionOperatorP1: Boundary treatment ...
00:04 DiffusionOperatorP1: Assemble system matrix ...
00:04 L2ProductP1: Integrate the products of the shape functions on each element
00:04 L2ProductP1: Determine global dofs ...
00:04 L2ProductP1: Boundary treatment ...
00:04 L2ProductP1: Assemble system matrix ...
00:05 DiffusionOperatorP1: Calculate gradients of shape functions transformed by reference map ...
00:05

Before we start solving the optimization problem, we first visualize the diffusion for a fixed parameter and the objective function. These are Figures 3 and 4 in the thesis. 

In [ ]:
from pymor.discretizers.builtin.cg import InterpolationOperator

mu_init = [0.25, 0.5]
mu_init = problem.parameters.parse(mu_init)

diff = InterpolationOperator(data['grid'], problem.diffusion).as_vector(mu_init)
fom.visualize(diff)

In [ ]:
from contour_plot_objective_function import plot_3d_surface, fom_objective_functional

ranges = parameter_space.ranges['diffusion']
XX = np.linspace(ranges[0] + 0.05, ranges[1], 50)
plot_3d_surface(fom, fom_objective_functional, XX, XX)

In the next step, we want to solve this optimization problem with a state-of-the art optimization algorithm, namely with the BFGS algorithm

In [ ]:
from two_dim_optimization_problem import fom_objective_functional, fom_gradient_of_functional, optimize_BFGS, report

amount_of_iters = 10
ranges = parameter_space.ranges['diffusion']
data = optimize_BFGS(fom_objective_functional, ranges, amount_of_iters, fom, fom_gradient_of_functional)
report(data, amount_of_iters)
np.save('reference_mu.npy', data['mu'])

In [ ]:
mu_ref_list = np.load('reference_mu.npy', allow_pickle=True)
print(mu_ref_list)

We will now utilize the proposed kernel TR algorithm to solve the optimization problem again. Before we compare the results with the BFGS algorithm from above, we compare the TRs of the standard and the advanved formulation. The outcome is Figure 5 in the thesis.

In [ ]:
from trust_region_kernel_approx import TR_Kernel, compute_RKHS_norm, draw_TR_standard
from vkoga.kernels import Gaussian, IMQ, Wendland, Matern

ep=1
global_counter = 0
mu_k = [0.25, 0.5]
mu_k = problem.parameters.parse(mu_k)

kernel = IMQ(ep=ep)
RKHS_norm, _ , _ = compute_RKHS_norm(kernel, fom, parameter_space)

TR_parameters={'radius': 0.5, 'sub_tolerance': 1e-5, 'max_iterations': 35, 'max_iterations_subproblem': 100,'starting_parameter': mu_k, 
               'max_iterations_armijo': 40, 'initial_step_armijo': 0.5, 'armijo_alpha': 1e-4, 'FOC_tolerance': 1e-8, 'J_tolerance': 1e-10,
               'beta_1': 0.5, 'beta_2': 0.95, 'rho': 0.9, 'eps': 0.03, 'max_amount_interpolation_points': 8, 'advanced': False, 'draw_TR': True}

mu_list, _ , list_delta = TR_Kernel(fom, kernel, parameter_space, global_counter, RKHS_norm, TR_parameters)
draw_TR_standard(list_delta, mu_list)





In [ ]:
from trust_region_kernel_approx import draw_TR_advanced
global_counter = 0

TR_parameters={'radius': 2, 'sub_tolerance': 1e-5, 'max_iterations': 10, 'max_iterations_subproblem': 100,'starting_parameter': mu_k, 
               'max_iterations_armijo': 40, 'initial_step_armijo': 0.5, 'armijo_alpha': 1e-4, 'FOC_tolerance': 1e-8, 'J_tolerance': 1e-10,
               'beta_1': 0.5, 'beta_2': 0.95, 'rho': 0.9, 'eps': 0.03, 'max_amount_interpolation_points': 8, 'width_gauss': ep, 'advanced': True, 'draw_TR': True}

mu_list,TR_plot_matrix , _ = TR_Kernel(fom, kernel, parameter_space, global_counter, RKHS_norm, TR_parameters)
draw_TR_advanced(TR_plot_matrix, mu_list)

In [ ]:
from trust_region_kernel_approx import optimize_all, report_kernel_TR, TR_Kernel

amount_of_iters = 10
#gamma_list = [m.sqrt(0.8), m.sqrt(0.9), m.sqrt(1), m.sqrt(1.1), m.sqrt(1.2)]
gamma_list = [1]
kernel_name = 'gauss'

TR_parameters={'radius': 0.5, 'sub_tolerance': 1e-5, 'max_iterations': 35, 'max_iterations_subproblem': 100, 'max_iterations_armijo': 40, 
            'initial_step_armijo': 0.5, 'armijo_alpha': 1e-4, 'FOC_tolerance': 1e-8, 'J_tolerance': 1e-10,
            'beta_1': 0.5, 'beta_2': 0.95, 'rho': 0.9, 'max_amount_interpolation_points': 8,  'advanced': False, 'draw_TR': False}

data = optimize_all(fom, parameter_space, TR_Kernel, kernel_name, gamma_list, TR_parameters, amount_of_iters)

report_kernel_TR(data, gamma_list, amount_of_iters)

In [ ]:
report_kernel_TR(data, gamma_list, amount_of_iters)

In [ ]:
mu_ref_list = np.load('reference_mu.npy', allow_pickle=True)
print(mu_ref_list)

In [ ]:
print(data)